## Tema 8 y 9: Análisis Discriminantes Lineal y Cuadrático

In [1]:
#Importando librerías necesarias
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

%matplotlib inline

### Cargar los datos

Porcentajes diarios de rendimiento para el índice bursátil S&P 500 entre 2001 y 2005.

- **Year**: el año en que se registró la observación.
- **Lag1**: porcentaje de devolución del día anterior
- **Lag2**: porcentaje de devolución de 2 días anteriores
- **Lag3**: Porcentaje de devolución de 3 días anteriores
- **Lag4**: porcentaje de devolución de 4 días anteriores
- **Lag5**: porcentaje de devolución de 5 días anteriores
- **Volume**: volumen de acciones negociadas (número de acciones diarias negociadas en miles de millones)
- **Today**: porcentaje de retorno para hoy
- **Direction**: un factor con niveles hacia abajo y hacia arriba que indica si el mercado tuvo un rendimiento positivo o negativo en un día determinado

En Python, podemos ajustar un modelo LDA usando la función LinearDiscriminantAnalysis(), que es parte del módulo discriminant_analysis de la librería de ML sklearn. Ajustaremos el modelo utilizando solo las observaciones anteriores a 2005, y luego probaremos el modelo con los datos de 2005.

In [2]:
#Estableciendo nuestra carpeta de trabajo
os.chdir(r"D:\Especializacion Python 19\Data")

In [3]:
#Importando nuestro archivo de trabajo
miarchivo="Smarket.csv"
df = pd.read_csv(miarchivo, 
                 usecols=range(1,10),
                 index_col=0, #La columna de índice 0 será utilizada como row labels
                 parse_dates=True) #Analiza gramaticalmente las fechas
df.head()

,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,
2001-01-01,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [4]:
type(df)

pandas.core.frame.DataFrame

## LDA

In [5]:
xtrain=df[:'2004'][['Lag1','Lag2']]
ytrain=df[:'2004']['Direction']
xtest=df[:'2005'][['Lag1','Lag2']]
ytest=df[:'2005']['Direction']
#Copia Adicional
y_train2 = df[:'2004']['Direction']

In [6]:
#Instanciamos un objeto de clase LinearDiscriminantAnalysis()
lda = LinearDiscriminantAnalysis()

#Hacer que el modelo aprenda de los datos
model = lda.fit(xtrain,# los valores de los predictores de entrenamiento
                ytrain)#los valores de y de entrenamiento

print("Las probabilidades a priori son:\n")
print("Probabilidad a Priori grupo I:",model.priors_[0].round(3))
print("Probabilidad a Priori grupo II:",model.priors_[1].round(3))

Las probabilidades a priori son:

Probabilidad a Priori grupo I: 0.492
Probabilidad a Priori grupo II: 0.508


In [7]:
#Contabilizando las frecuencias de la target
pd.value_counts(ytrain)

Up      507
Down    491
Name: Direction, dtype: int64

La salida LDA indica probabilidades previas de $\hat{π}_1=0.492$ y $\hat{π}_2=0.508$; en otras palabras, el 49.2% de las observaciones de entrenamiento corresponden a días durante los cuales el mercado Down.

In [8]:
print("Los centroides de los grupos son:\n")
print("Centroide del grupo I:" ,model.means_[0])
print("Centroide del grupo II:" ,model.means_[1])

Los centroides de los grupos son:

Centroide del grupo I: [0.04279022 0.03389409]
Centroide del grupo II: [-0.03954635 -0.03132544]


Lo anterior proporciona los medios grupales; estos son el promedio de cada predictor dentro de cada clase, y son utilizados por LDA como estimaciones de  $μ_k$. Esto sugiere que existe una tendencia a que los retornos de los 2 días anteriores sean negativos en los días en que el mercado aumenta, y una tendencia a que los retornos de los días anteriores sean positivos en los días en que el mercado declina.

In [9]:
print("Los coeficientes estimados para la Función Discriminante es:")
print(model.coef_)

Los coeficientes estimados para la Función Discriminante es:
[[-0.05544078 -0.0443452 ]]


Los coeficientes de salida discriminantes lineales proporcionan la combinación lineal de Lag1y Lag2 que se utilizan para formar la regla de decisión LDA.

$If −0.0554×Lag1−0.0443×Lag2$ es grande, entonces el clasificador LDA predecirá un aumento del mercado, y si es pequeño, entonces el clasificador LDA pronosticará una disminución del mercado. 



In [10]:
pred=model.predict(xtest)
print(np.unique(pred, 
                return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([398, 852], dtype=int64))


El modelo asignó 70 observaciones a la clase "Down" y 182 observaciones a la clase "Up". Veamos la matriz de confusión para ver cómo le está yendo a este modelo. Queremos comparar la clase predicha (que podemos encontrar en pred) con la clase verdadera (que se encuentra en $y_test$)

### Matriz de confusión datos de entrenamiento

In [11]:
pred2=model.predict(xtrain)
#pred2

In [12]:
print(np.unique(pred2, return_counts=True))

(array(['Down', 'Up'], dtype='<U4'), array([328, 670], dtype=int64))


In [13]:
#Haciendo la matriz de confusión con confusion_matrix de sklearn
print(confusion_matrix(pred2, y_train2))

[[168 160]
 [323 347]]


In [14]:
print(classification_report(y_train2, #los valores reales
                            pred2, #los valores predichos
                            digits=2)) #digitos

              precision    recall  f1-score   support

        Down       0.51      0.34      0.41       491
          Up       0.52      0.68      0.59       507

    accuracy                           0.52       998
   macro avg       0.52      0.51      0.50       998
weighted avg       0.52      0.52      0.50       998



### Matriz de confusión datos de prueba

In [15]:
print(confusion_matrix(pred, ytest))

[[203 195]
 [399 453]]


In [16]:
confusion_matrix = pd.crosstab(pred, ytest)
confusion_matrix

Direction,Down,Up
row_0,,
Down,203,195
Up,399,453


In [17]:
print(classification_report(ytest, pred, digits=2))

              precision    recall  f1-score   support

        Down       0.51      0.34      0.41       602
          Up       0.53      0.70      0.60       648

    accuracy                           0.52      1250
   macro avg       0.52      0.52      0.51      1250
weighted avg       0.52      0.52      0.51      1250



## QDA

In [18]:
#Instanciando una clase QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()

In [19]:
#Aprendemos de los datos de entrenamiento
model2 = qda.fit(xtrain, #X DE ENTRENAMIENTO 
                 ytrain) #Y DE ENTRENAMIENTO
print(model2.priors_)

[0.49198397 0.50801603]


In [20]:
#Centroides de los grupos
print(model2.means_)

[[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]


### Matriz de confusión datos de entrenamiento

In [21]:
pred3=model2.predict(xtrain)
print(np.unique(pred3, return_counts=True))

(array(['Down', 'Up'], dtype=object), array([318, 680], dtype=int64))


In [22]:
print(classification_report(ytrain, pred3, digits=2))

              precision    recall  f1-score   support

        Down       0.51      0.33      0.40       491
          Up       0.52      0.69      0.59       507

    accuracy                           0.51       998
   macro avg       0.51      0.51      0.50       998
weighted avg       0.51      0.51      0.50       998



### Matriz de confusión datos de prueba

In [25]:
pred4=model2.predict(xtest)
print(np.unique(pred4, return_counts=True))

(array(['Down', 'Up'], dtype=object), array([368, 882], dtype=int64))


In [26]:
print(classification_report(ytest, pred4, digits=2))

              precision    recall  f1-score   support

        Down       0.52      0.32      0.40       602
          Up       0.54      0.73      0.62       648

    accuracy                           0.53      1250
   macro avg       0.53      0.52      0.51      1250
weighted avg       0.53      0.53      0.51      1250

